In [28]:
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine # database connectio
import os
import re
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
#from wordcloud import WordCloud

In [6]:
if not os.path.isfile('train.db'):
    start = dt.datetime.now()
    disk_engine = create_engine('sqlite:///train.db')
    start = dt.datetime.now()
    chunksize = 100000
    j = 0
    index_start = 1
    for df in pd.read_csv('Train.csv',names=['Id', 'Title', 'Body', 'Tags'], chunksize=chunksize, iterator=True, 
                          encoding='utf-8'):
        df.index += index_start
        j = j+1
        print('{}.rows'.format(j*chunksize))
        df.to_sql('data', disk_engine, if_exists='append')
        index_start = df.index[-1] + 1
    print('Time taken to insert data ,',dt.datetime.now() - start)

100000.rows
200000.rows
300000.rows
400000.rows
500000.rows
600000.rows
700000.rows
800000.rows
900000.rows
1000000.rows
1100000.rows
1200000.rows
1300000.rows
1400000.rows
1500000.rows
1600000.rows
1700000.rows
1800000.rows
1900000.rows
2000000.rows
2100000.rows
2200000.rows
2300000.rows
2400000.rows
2500000.rows
2600000.rows
2700000.rows
2800000.rows
2900000.rows
3000000.rows
3100000.rows
3200000.rows
3300000.rows
3400000.rows
3500000.rows
3600000.rows
3700000.rows
3800000.rows
3900000.rows
4000000.rows
4100000.rows
4200000.rows
4300000.rows
4400000.rows
4500000.rows
4600000.rows
4700000.rows
4800000.rows
4900000.rows
5000000.rows
5100000.rows
5200000.rows
5300000.rows
5400000.rows
5500000.rows
5600000.rows
5700000.rows
5800000.rows
5900000.rows
6000000.rows
6100000.rows
Time taken to insert data , 0:05:04.904890


In [12]:
# counting number of row in data fike
if os.path.isfile('train.db'):
    con = sqlite3.connect('train.db')
    num_of_rows = pd.read_sql_query("""select count(*) from data""", con)
    print('number of rows in database are ',num_of_rows)
    con.close()
else:
    print('file not found')

number of rows in database are     count(*)
0   6034196


In [6]:
# counting number of row in data fike
if os.path.isfile('train.db'):
    start = dt.datetime.now()
    con = sqlite3.connect('train.db')
    num_of_rows = pd.read_sql_query("""select * from data limit 1000""", con)
    #print('number of rows in database are ',num_of_rows)
    con.close()
    print('Time required was',dt.datetime.now() - start)
else:
    print('file not found')

Time required was 0:00:00.015926


In [7]:
num_of_rows

,index,Id,Title,Body,Tags
0,1,Id,Title,Body,Tags
1,2,1,How to check if an uploaded file is an image w...,<p>I'd like to check if an uploaded file is an...,php image-processing file-upload upload mime-t...
2,3,2,How can I prevent firefox from closing when I ...,"<p>In my favorite editor (vim), I regularly us...",firefox
3,4,3,R Error Invalid type (list) for variable,<p>I am import matlab file and construct a dat...,r matlab machine-learning
4,5,4,How do I replace special characters in a URL?,"<p>This is probably very simple, but I simply ...",c# url encoding
...,...,...,...,...,...
995,996,995,TP Link wireless router connects but web pages...,"<p>i have recently purchased <a href=""http://r...",wireless-router tp-link mtu
996,997,996,Forbid a given user to use a given range of port,<p>I would like to forbid a user account on li...,linux
997,998,997,How to find which block displays content?,<p>I have to modify a website made by someone ...,block prestashop
998,999,998,Struct with `initializer_list` ctor inside union?,"<p>I have a struct which is POD, but for conve...",c++ c++11 initializer-list unions


In [8]:
# checking for duplicate records
if os.path.isfile('train.db'):
    start = dt.datetime.now()
    con = sqlite3.connect('train.db')
    #query = '''select Title, Body,Tags, count(*) as cnt_dup from data group by Title,Body,Tags'''
    df_with_dup = pd.read_sql_query("""select Title, Body,Tags, count(*) as cnt_dup from data GROUP BY Title,Body,Tags""",con)
    con.close()
    print('Time required was',dt.datetime.now() - start)

Time required was 0:23:40.414809


In [11]:
df_with_dup.cnt_dup.value_counts()

1    2656284
2    1272336
3     277575
4         90
5         25
6          5
Name: cnt_dup, dtype: int64

In [24]:
no_dup =( (num_of_rows['count(*)'].values[0]-df_with_dup.shape[0]) / num_of_rows['count(*)'].values[0]) *100
print('Percet of duplicate questions is,',no_dup,'%')

Percet of duplicate questions is, 30.292038906260256 %


In [35]:
df_with_dup.isnull().sum()

Title      0
Body       0
Tags       7
cnt_dup    0
dtype: int64

In [36]:
# dropping null values
df_with_dup.dropna(inplace=True)

In [37]:
# creating a new coln to count number of tags for each row
df_with_dup['cnt_tags'] = df_with_dup.Tags.apply(lambda x:len(x.split(" ")))

In [40]:
# checking distribuition of tags per question
df_with_dup.cnt_tags.value_counts()

3    1206157
2    1111706
4     814996
1     568291
5     505158
Name: cnt_tags, dtype: int64

In [ ]:
# creating new database with no duplicate data 
if not os.path.isfile('train_with_no_duplicate.db'):
    disk_dup = create_engine("sqlite:///train_with_no_duplicate.db")
    no_duplicate = pd.DataFrame(df_with_dup,columns=['Title','Body','Tags'])
    no_duplicate.to_sql('no_duplicate_data',disk_dup)
else:
    print('no created')

In [45]:
if os.path.isfile('train_with_no_duplicate.db'):
    start = datetime.now()
    con = sqlite3.connect('train_with_no_duplicate.db')
    tag_data = pd.read_sql_query("""SELECT Tags FROM no_duplicate_data""", con)
    #Always remember to close the database
    con.close()

    # Let's now drop unwanted column.
    tag_data.drop(tag_data.index[0], inplace=True)
    #Printing first 5 columns from our data frame
    tag_data.head()
    print("Time taken to run this cell :", datetime.now() - start)
else:
    print("Please download the train.db file from drive or run the above cells to genarate train.db file")

Please download the train.db file from drive or run the above cells to genarate train.db file
